# Heads Up

In this notebook, we start generating random undirected weighted
graphs to not only better approximate real world road network but
also facilitate the feature engineering of nodes and edges

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import random
import numpy as np
import mxnet as mx
import logging
from sklearn.metrics import accuracy_score
from utils import plot_g

In [ ]:
NUM_NODE = 100
WEIGHT_MIN = .0
WEIGHT_MAX = 1.

In [ ]:
def generate_low_degree_g(node_size=20, min_out_degree=2, max_out_degree=4, weight_min=WEIGHT_MIN, weight_max=WEIGHT_MAX):
    
    G = nx.Graph()
    G.add_nodes_from(range(0, node_size))
    
    for node in G.nodes:
        
        tmp_nodes = list(G.nodes)
        tmp_nodes.remove(node)
        random.shuffle(tmp_nodes)
        num_of_neighbors = random.randint(min_out_degree, max_out_degree)
#         print(node, out_neighbors)
        
#         G.add_edges_from(map(lambda d:(node, d), out_neighbors))
        
        for tmp_node in tmp_nodes:

            if G.degree(tmp_node) >= max_out_degree \
                or G.degree(node) >= num_of_neighbors:
                # This node has maximum number of neighbors already
                continue
            
            weight = random.uniform(weight_min, weight_max)
            G.add_edge(node, tmp_node, weight=weight)
        
    return G

In [ ]:
G = generate_low_degree_g()
max_degree = max(G.degree, key=lambda d: d[1])[1]
min_degree = min(G.degree, key=lambda d: d[1])[1]
print(max_degree)
print(min_degree)
print(G.number_of_nodes())
print(G.number_of_edges())
plot_g(G)

In [ ]:
def traverse_g_helper(layer_2_layer_dict, G, parent_node, cur_node, step, depth):
    
    if step > depth:
        return
    
    layer_2_layer_dict[step-1][(parent_node, cur_node)] = \
                        filter(lambda edge: edge[1] != parent_node, G.edges(cur_node))
        
    for edge in G.edges(cur_node):
        
        neighbor = edge[1]
        if neighbor == parent_node:
            continue
        
#         print('Step %d: from %d to %d' % (step, cur_node, neighbor))
        traverse_g_helper(layer_2_layer_dict, G, cur_node, neighbor, step+1, depth)
    
def traverse_g(G, src, parent_node=-1, depth=3):
    
    layer_2_layer_dict = [dict() for _ in range(depth)]
    traverse_g_helper(layer_2_layer_dict, G, parent_node, src, 1, depth)
    
    return layer_2_layer_dict

In [ ]:
DST_NEIGHBOR_LAYERS = generate_dst_neighbor_layers(G)
PAIR_NEIGHBOR_LAYERS = generate_pair_neighbor_layers(G)

In [325]:
DST_NEIGHBOR_LAYERS[0]

/usr/local/Cellar/python/2.7.14/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2018-03-11 23:40:36.840932. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


[{(-1, 0): [(0, 2), (0, 19), (0, 4), (0, 5)]},
 {(0, 2): [(2, 11), (2, 3), (2, 7)],
  (0, 4): [(4, 18), (4, 6), (4, 14)],
  (0, 5): [(5, 9), (5, 6)],
  (0, 19): [(19, 16), (19, 18), (19, 12)]},
 {(2, 3): [(3, 16), (3, 10), (3, 7)],
  (2, 7): [(7, 1), (7, 3)],
  (2, 11): [(11, 1), (11, 10)],
  (4, 6): [(6, 5), (6, 14)],
  (4, 14): [(14, 8), (14, 18), (14, 6)],
  (4, 18): [(18, 19), (18, 14)],
  (5, 6): [(6, 4), (6, 14)],
  (5, 9): [(9, 15)],
  (19, 12): [(12, 8), (12, 13)],
  (19, 16): [(16, 1), (16, 3), (16, 17)],
  (19, 18): [(18, 4), (18, 14)]}]

In [326]:
def encode_layers(G, layers, end=None, depth=3):
    
    assert type(layers) is list and len(layers) == depth
    ret = [ np.ones(max_out_degree ** (i+1)) for i in range(depth)]
    label = None
    
    for layer in range(depth):
        
        assert type(layers[layer]) is dict
        
        cnt = 0
        
        for _, edge_list in layers[layer].items():
            
            assert type(edge_list) is list
            
            for idx, edge in enumerate(edge_list):
                ret[layer][cnt] = G.get_edge_data(edge[0], edge[1])['weight']
                cnt += 1
                
                if layer == 0 and end is not None:
                    if end == edge[1]:
                        label = idx
        
    return reduce(lambda x, y: np.hstack([x, y]), ret), label

def encode_sample(G, start, end, dst, depth=3):
    
    dst_layers_encoded, _ = encode_layers(G, DST_NEIGHBOR_LAYERS[dst])
    edge_layers_encoded, label = encode_layers(G, DST_NEIGHBOR_LAYERS[start], end=end)
    
    return np.hstack([edge_layers_encoded, dst_layers_encoded]), label

X = []
y = []

for paths in nx.all_pairs_dijkstra_path(G):
    print("src :%d" % paths[0])
    print(paths[1])
    print('')
    src = paths[0]
    for dst, path in paths[1].items():
        
        if dst == src:
            continue
            
        # X = cur_node + dst_node
        # y = next_node (Dijkstra)
        
        cur_node = src
        for mid_node in path[1:]:
            
            print('X:(%d, %d), y:(%d)' % (cur_node, dst, mid_node))
            feature, label = encode_sample(G, cur_node, mid_node, dst)
            X.append(feature)
            y.append(label)
            
            cur_node = mid_node
            
            # Retrieve edge(cur_node, mid_node) || dst
            
#         print(cur_node, dst)
        print(path)
        print('')

/usr/local/Cellar/python/2.7.14/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2018-03-11 23:46:45.421698. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


src :0
{0: [0], 1: [0, 2, 3, 7, 1], 2: [0, 2], 3: [0, 2, 3], 4: [0, 5, 6, 4], 5: [0, 5], 6: [0, 5, 6], 7: [0, 2, 3, 7], 8: [0, 19, 18, 14, 8], 9: [0, 5, 9], 10: [0, 2, 3, 10], 11: [0, 2, 11], 12: [0, 19, 12], 13: [0, 19, 12, 13], 14: [0, 19, 18, 14], 15: [0, 5, 9, 15], 16: [0, 19, 16], 17: [0, 19, 16, 17], 18: [0, 19, 18], 19: [0, 19]}

X:(0, 1), y:(2)
X:(2, 1), y:(3)
X:(3, 1), y:(7)
X:(7, 1), y:(1)
[0, 2, 3, 7, 1]

X:(0, 2), y:(2)
[0, 2]

X:(0, 3), y:(2)
X:(2, 3), y:(3)
[0, 2, 3]

X:(0, 4), y:(5)
X:(5, 4), y:(6)
X:(6, 4), y:(4)
[0, 5, 6, 4]

X:(0, 5), y:(5)
[0, 5]

X:(0, 6), y:(5)
X:(5, 6), y:(6)
[0, 5, 6]

X:(0, 7), y:(2)
X:(2, 7), y:(3)
X:(3, 7), y:(7)
[0, 2, 3, 7]

X:(0, 8), y:(19)
X:(19, 8), y:(18)
X:(18, 8), y:(14)
X:(14, 8), y:(8)
[0, 19, 18, 14, 8]

X:(0, 9), y:(5)
X:(5, 9), y:(9)
[0, 5, 9]

X:(0, 10), y:(2)
X:(2, 10), y:(3)
X:(3, 10), y:(10)
[0, 2, 3, 10]

X:(0, 11), y:(2)
X:(2, 11), y:(11)
[0, 2, 11]

X:(0, 12), y:(19)
X:(19, 12), y:(12)
[0, 19, 12]

X:(0, 13), y:(19)
X:(19, 

In [338]:
print(y[2])
DST_NEIGHBOR_LAYERS[3]

3


/usr/local/Cellar/python/2.7.14/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2018-03-11 23:50:24.946811. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


[{(-1, 3): [(3, 16), (3, 2), (3, 10), (3, 7)]},
 {(3, 2): [(2, 0), (2, 11), (2, 7)],
  (3, 7): [(7, 1), (7, 2)],
  (3, 10): [(10, 11), (10, 15)],
  (3, 16): [(16, 19), (16, 1), (16, 17)]},
 {(2, 0): [(0, 19), (0, 4), (0, 5)],
  (2, 7): [(7, 1), (7, 3)],
  (2, 11): [(11, 1), (11, 10)],
  (7, 1): [(1, 16), (1, 11)],
  (7, 2): [(2, 0), (2, 11), (2, 3)],
  (10, 11): [(11, 1), (11, 2)],
  (10, 15): [(15, 8), (15, 9)],
  (16, 1): [(1, 11), (1, 7)],
  (16, 17): [(17, 13)],
  (16, 19): [(19, 0), (19, 18), (19, 12)]}]

In [ ]:
X = np.array(X)
y = np.array(y)

print(X.shape)
print(y.shape)
y.tolist().count(3)

In [ ]:
def generate_dst_neighbor_layers(G, depth=3):
    
    ret = dict()
    for node in G.nodes:
        ret[node] = traverse_g(G, node, parent_node=-1, depth=depth)
    
    return ret

def generate_pair_neighbor_layers(G, depth=3):
    
    ret = dict()
    for edge in G.edges:
        ret[(edge[0], edge[1])] = traverse_g(G, edge[1], parent_node=edge[0], depth=depth)
        ret[(edge[1], edge[0])] = traverse_g(G, edge[0], parent_node=edge[1], depth=depth)
    
    return ret

def generate_dataset(G, depth=3):
    
    pass